<a href="https://colab.research.google.com/github/rgmartin/1-logistic_regression_classifier/blob/main/greece_tourism_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Greece tourism detination analysis



## 1. Web scrapping excel files

In [1]:
! pip install plotly --upgrade

     |████████████████████████████████| 23.9 MB 12 kB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [2]:
#####Step 1: start by importing all of the necessary packages#####
import requests #requesting URLs
import urllib.request #requesting URLs
import os
import time #setting the speed at which the requests run
import re #regexp string manipulation
import pandas as pd #for simplifying data operations (e.g. creating dataframe objects)
import matplotlib.pyplot as plt #for plotting the scraped data
from bs4 import BeautifulSoup #for web-scraping operations

In [3]:
#####Step 2: connect to the URL in question for scraping#####
url = 'https://insete.gr/perifereies/' 
response = requests.get(url) #Connect to the URL using the "requests" package
response #if successful then it will return 200
#####Step 3: read in the URL via the "BeautifulSoup" package#####
soup = BeautifulSoup(response.text, 'html.parser')
#print(soup)
#####Step 4: filter the HTML object for all link objects######
#link_objs = soup.find_all('a', href=re.compile('/wp-content/uploads'))
excel_links = [link.get('href') for link in soup.find_all('a') if ('xlsx' in link.get('href')) ]

In [4]:
df_parameters= [
                          {'sheet_name': 6, 'header':[2,3],'index_col':[0,1]}

]

In [5]:
def extract_region_name(url):
  file_w_extension = os.path.basename(url)
  file = os.path.splitext(file_w_extension)[0]  
  return ''.join(x for x in file if x.isalpha())

In [6]:
def read_inbound_tourism(url):
  data = requests.get(url).content
  xl = pd.ExcelFile(data)
  parameters = {'sheet_name': 6, 'header':[3],'index_col':[0,1]}
  df = xl.parse(**parameters)
  series = df.loc['Σύνολο'].iloc[0]
  series.name = extract_region_name(url)
  return series

In [95]:
def read_inbound_tourism_per_country(url, key_figure = 0):
  data = requests.get(url).content
  xl = pd.ExcelFile(data)
  parameters = {'sheet_name': 2, 'header':[3],'index_col':[0,1]}
  # localise those rows corresponding to the first multindex
  df = xl.parse(**parameters)  
  region_name = extract_region_name(url)
  second_level_last_index = list(df.index.get_level_values(1)).index('Σύνολο') 
  df = df.iloc[ :second_level_last_index,key_figure].droplevel(level=0)
  #df.rename(columns={ df.columns[1]: region_name }, inplace = True)
  df.name = region_name
  return df
  

In [97]:
inbound_tourism_df = pd.concat([read_inbound_tourism_per_country(url) for url in excel_links], axis = 1)
inbound_tourism_df
#inbound_tourism_df.index.name = 'Year'

,EasternMacedoniaThrace,Attica,NorthAegean,WesternGreece,WesternMacedonia,Epirus,Thessaly,IonianIslands,CentralMacedonia,Crete,SouthAegean,Peloponnese,CentralGreece
Τουρκία,123.101,75.203,1.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Βουλγαρία,316.427,NaN,NaN,NaN,6.635,NaN,NaN,NaN,205.382,NaN,NaN,NaN,NaN
Γερμανία,32.999,180.94,9.549,24.343,13.691,22.857,25.069,141.648,249.118,449.847,441.199,46.154,17.099
Ρουμανία,58.249,NaN,NaN,NaN,1.226,NaN,5.916,NaN,66.704,NaN,NaN,NaN,NaN
Λοιπές,71.05,667.734,27.516,91.961,21.208,146.635,100.961,144.637,361.787,306.255,389.175,174.383,85.446
Κύπρος,NaN,180.257,NaN,NaN,NaN,NaN,NaN,NaN,44.555,NaN,NaN,NaN,NaN
Ην. Βασίλειο,NaN,200.309,5.718,17.947,NaN,11.389,30.377,275.976,98.134,149.869,313.248,29.348,20.252
Γαλλία,NaN,139.082,NaN,13.127,NaN,NaN,NaN,33.207,NaN,149.582,153.583,28.207,25.17
ΗΠΑ,NaN,81.046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ιταλία,NaN,97.074,NaN,NaN,NaN,19.122,13.691,80.039,NaN,38.019,95.194,NaN,NaN


In [117]:
import matplotlib.pyplot as plt
import matplotlib

cm = plt.get_cmap('gist_rainbow')
n = 10
colors = [matplotlib.colors.rgb2hex(rgba) for rgba in cm(range(n))]
print(colors)

['#ff0029', '#ff0023', '#ff001e', '#ff0019', '#ff0013', '#ff000e', '#ff0009', '#ff0003', '#ff0200', '#ff0700']


In [134]:
#TODO FINISH the Sankey Diagram
import matplotlib.pyplot as plt

def nodes_df(df):
  regions = list(df.columns)
  countries = list(df.index)
  df = pd.DataFrame(data = regions + countries, columns = ['Label'])
  df.index.name = 'ID'

  cm = plt.get_cmap('gist_rainbow')
  n = len(regions  + countries)
  colors = [matplotlib.colors.rgb2hex(rgba) for rgba in cm(range(n))]

  df['Color'] = colors

  return df  

def links_df(df):
  list_of_targ_detinations_pairs = list(inbound_tourism_df[inbound_tourism_df.notnull()].stack().index)

In [137]:
list(inbound_tourism_df[inbound_tourism_df.notnull()].stack().index)

[('Τουρκία', 'EasternMacedoniaThrace'),
 ('Τουρκία', 'Attica'),
 ('Τουρκία', 'NorthAegean'),
 ('Βουλγαρία', 'EasternMacedoniaThrace'),
 ('Βουλγαρία', 'WesternMacedonia'),
 ('Βουλγαρία', 'CentralMacedonia'),
 ('Γερμανία', 'EasternMacedoniaThrace'),
 ('Γερμανία', 'Attica'),
 ('Γερμανία', 'NorthAegean'),
 ('Γερμανία', 'WesternGreece'),
 ('Γερμανία', 'WesternMacedonia'),
 ('Γερμανία', 'Epirus'),
 ('Γερμανία', 'Thessaly'),
 ('Γερμανία', 'IonianIslands'),
 ('Γερμανία', 'CentralMacedonia'),
 ('Γερμανία', 'Crete'),
 ('Γερμανία', 'SouthAegean'),
 ('Γερμανία', 'Peloponnese'),
 ('Γερμανία', 'CentralGreece'),
 ('Ρουμανία', 'EasternMacedoniaThrace'),
 ('Ρουμανία', 'WesternMacedonia'),
 ('Ρουμανία', 'Thessaly'),
 ('Ρουμανία', 'CentralMacedonia'),
 ('Λοιπές', 'EasternMacedoniaThrace'),
 ('Λοιπές', 'Attica'),
 ('Λοιπές', 'NorthAegean'),
 ('Λοιπές', 'WesternGreece'),
 ('Λοιπές', 'WesternMacedonia'),
 ('Λοιπές', 'Epirus'),
 ('Λοιπές', 'Thessaly'),
 ('Λοιπές', 'IonianIslands'),
 ('Λοιπές', 'CentralMacedo

In [135]:
create_sankey_diagram(inbound_tourism_df)

,Label,Color
ID,,
0,EasternMacedoniaThrace,#ff0029
1,Attica,#ff0023
2,NorthAegean,#ff001e
3,WesternGreece,#ff0019
4,WesternMacedonia,#ff0013
5,Epirus,#ff000e
6,Thessaly,#ff0009
7,IonianIslands,#ff0003
8,CentralMacedonia,#ff0200


In [99]:
import plotly.graph_objects as go
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = ["A1", "A2", "B1", "B2", "C1", "C2"],
      color = "blue"
    ),
    link = dict(
      source = [0, 1, 0, 2, 3, 3], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = [2, 3, 3, 4, 4, 5],
      value = [8, 4, 2, 8, 4, 2]
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()

In [7]:
inbound_tourism_df = pd.concat([read_inbound_tourism(url) for url in excel_links], axis = 1)
inbound_tourism_df.index.name = 'Year'

In [8]:
inbound_tourism_df.sort_values(axis=1, by=2020,ascending=False,inplace=True)
inbound_tourism_shares = inbound_tourism_df.div(inbound_tourism_df.sum(axis=1), axis=0)*100
inbound_tourism_shares

,Crete,Attica,IonianIslands,CentralMacedonia,SouthAegean,Peloponnese,Thessaly,EasternMacedoniaThrace,CentralGreece,WesternGreece,Epirus,NorthAegean,WesternMacedonia
Year,,,,,,,,,,,,,
2010,28.260664,27.002760,11.027154,11.641536,4.186582,4.278634,3.167388,1.594131,2.305475,3.513032,0.807280,1.936322,0.279043
2011,30.331411,25.424492,10.808777,11.862583,4.361663,3.957447,3.027026,1.780052,2.148323,3.180805,0.808558,2.008231,0.300633
2012,32.842341,22.338077,11.745774,13.605866,4.078317,3.245820,2.709035,2.007416,1.626473,2.608993,0.838479,2.058555,0.294854
2013,32.568554,22.518324,11.248665,13.480498,4.505238,2.916547,2.859138,2.420278,1.560248,2.557135,0.816664,2.272205,0.276506
2014,30.336098,25.405419,10.991240,12.653784,5.097656,3.209608,2.985335,2.413795,1.729735,1.948138,0.810574,2.229178,0.189440
2015,28.701265,25.171607,11.459370,12.944135,5.312086,3.131549,3.001081,2.603866,1.796053,2.508087,0.796729,2.417224,0.156948
2016,31.377831,23.809769,12.186710,12.594433,5.288574,3.272879,2.515891,2.529223,1.786255,1.850952,0.882655,1.768303,0.136526
2017,29.991054,23.850993,12.424858,12.115419,5.570214,3.813966,2.888588,2.433445,2.003214,1.900148,0.976197,1.878976,0.152926
2018,29.631345,21.649635,12.375822,11.636936,8.345736,4.154666,3.003820,2.187099,2.249706,1.838447,1.147458,1.633175,0.146155


In [9]:
# visualisation based on https://towardsdatascience.com/create-effective-data-visualizations-of-proportions-94b69ad34410
# ideas of visualisation https://www.politico.eu/article/europe-tourism-boom-time-overtourism-top-destinations/
import plotly.express as px
fig = px.bar(inbound_tourism_shares,x = inbound_tourism_shares.index, y =inbound_tourism_shares.columns)
fig.show()

In [ ]:
url= 'https://insete.gr/wp-content/uploads/2020/05/21-04_Eastern_Macedonia__Thrace-3.xlsx'
data = requests.get(url).content
xl = pd.ExcelFile(data)
xl.sheet_names

## Relevant questions
* Has the Greece tourism industry been devastated by COVID restrictions?
* How the origin country distribution was affected by COVID?
  - Plot a graph of shares of visitors from different countries (see https://www.kaggle.com/ceshine/impact-of-chinese-tourism-ban-to-taiwan)
* Analysis of seasonality
* Draw insights on how tourism has changed from different countries over time.
* Use other world datasets to draw the possible reasons/factors affecting tourism.
* domestic arrivals vs international arrivals vs road arrivals

TODO: create function that is able to read any specific data frame set from the excel documents. 
- Compare the share of tourism for each region.!

## Growth rate of tourism spending by international and domestic tourists

## Population of region vs inbound tourism

## Tourism origin country